In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/ZetaGroup1/WeatherMachine/refs/heads/main/Final_data_scripts/Data/predicted_snow_conditions.csv')

df

,Date,cloud_cover_code,Precipitation amount [mm],Snow depth [cm],Average temperature [°C],Direct solar radiation mean [W/m2],days_above_10cm,days_above_20cm,Predicted Snow Depth [cm]
0,2005-01-02,3.0,4.2,15.0,0.2,1.1,1,0,13.891783
1,2005-01-03,3.0,0.8,19.0,-1.4,0.7,2,0,15.793369
2,2005-01-04,3.0,1.8,19.0,-1.8,1.8,3,0,17.248934
3,2005-01-05,3.0,6.2,20.0,-1.5,1.8,4,0,17.672102
4,2005-01-06,3.0,5.0,27.0,-1.3,1.7,5,1,17.215079
...,...,...,...,...,...,...,...,...,...
29221,2024-12-28,1.0,-1.0,31.0,-2.9,1.5,33,13,18.218463
29222,2024-12-29,8.0,1.9,31.0,-0.7,0.5,34,14,19.594800
29223,2024-12-30,3.0,0.8,32.0,-5.3,42.0,35,15,25.966975
29224,2024-12-31,8.0,-1.0,34.0,-2.2,61.7,36,16,23.966696


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29226 entries, 0 to 29225
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Date                                29226 non-null  object 
 1   cloud_cover_code                    29226 non-null  float64
 2   Precipitation amount [mm]           29226 non-null  float64
 3   Snow depth [cm]                     29226 non-null  float64
 4   Average temperature [°C]            29226 non-null  float64
 5   Direct solar radiation mean [W/m2]  29226 non-null  float64
 6   days_above_10cm                     29226 non-null  int64  
 7   days_above_20cm                     29226 non-null  int64  
 8   Predicted Snow Depth [cm]           29226 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 2.0+ MB


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

for year in range(2005, 2025):
    # Filter the dataframe for the specific year
    df_year = df[df['Date'].dt.year == year]
    
    # --- Check for continuous 14 days of snow with at least 20 cm ---
    snow_20cm = df_year[df_year['Snow depth [cm]'] >= 20].copy()  # Use .copy() to avoid the warning
    snow_20cm['date_diff'] = snow_20cm['Date'].diff().dt.days.fillna(1)  # Calculate the difference in days between rows
    snow_20cm['group'] = (snow_20cm['date_diff'] > 1).cumsum()  # Group consecutive snow days
    
    # Count how many consecutive days in each group
    consecutive_20cm = snow_20cm.groupby('group').size()
    result_14_days_20cm = any(consecutive_20cm >= 14)  # If any group has 14 or more days

    # --- Check for continuous 28 days of snow with at least 10 cm ---
    snow_10cm = df_year[df_year['Snow depth [cm]'] >= 10].copy()  # Use .copy() to avoid the warning
    snow_10cm['date_diff'] = snow_10cm['Date'].diff().dt.days.fillna(1)
    snow_10cm['group'] = (snow_10cm['date_diff'] > 1).cumsum()
    
    # Count how many consecutive days in each group
    consecutive_10cm = snow_10cm.groupby('group').size()
    result_28_days_10cm = any(consecutive_10cm >= 28)  # If any group has 28 or more days
    
    # Print the results for this year
    print(f"Year {year}:")
    print(f"  Continuous 14 days of snow with at least 20 cm: {result_14_days_20cm}")
    print(f"  Continuous 28 days of snow with at least 10 cm: {result_28_days_10cm}")
    print()